# Week 2 — LoRA Fine-Tuning of SLM
Align Small Language Model embeddings with Diffusion CLIP Encoder.

In [1]:
!pip install peft==0.10.0 transformers==4.37.2 diffusers==0.27.2 accelerate==0.29.3 -q

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from diffusers import StableDiffusionPipeline
from peft import LoraConfig, get_peft_model, TaskType
import torch, pandas as pd
from tqdm import tqdm

/home/cse-sdpl/anaconda3/envs/gpu_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]


ImportError: cannot import name 'cached_download' from 'huggingface_hub' (/home/cse-sdpl/anaconda3/envs/gpu_env/lib/python3.10/site-packages/huggingface_hub/__init__.py)

In [ ]:
slm_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(slm_name)
slm = AutoModelForCausalLM.from_pretrained(slm_name, torch_dtype=torch.float16).to("cuda")
pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16).to("cuda")
clip_encoder = pipe.text_encoder
print("SLM & Diffusion Model Loaded Successfully")

In [ ]:
data = {"caption": [
    "a sunset over the ocean",
    "a cat sitting on a laptop",
    "a futuristic city skyline",
    "a person walking in rain with umbrella",
] * 25}
df = pd.DataFrame(data)
df.to_csv("../data/mini_text_image_dataset.csv", index=False)
print("Dataset saved")

In [ ]:
config = LoraConfig(task_type=TaskType.CAUSAL_LM, r=8, lora_alpha=16, target_modules=["q_proj", "v_proj"], lora_dropout=0.05)
slm_lora = get_peft_model(slm, config)
slm_lora.print_trainable_parameters()

In [ ]:
def dummy_train(slm, tokenizer, df):
    optimizer = torch.optim.AdamW(slm.parameters(), lr=1e-4)
    slm.train()
    for caption in tqdm(df['caption'][:50], desc="Training"):
        inputs = tokenizer(caption, return_tensors="pt", truncation=True, max_length=128).to("cuda")
        outputs = slm(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print("LoRA Fine-tuning Complete")
dummy_train(slm_lora, tokenizer, df)

In [ ]:
output_dir = "../adapters/slm_lora_adapter/"
slm_lora.save_pretrained(output_dir)
print(f"LoRA adapter saved to {output_dir}")